In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from keras.layers import Dense
from keras import layers
from copy import deepcopy as dp
from sklearn import metrics

In [ ]:
raw = pd.read_csv('../data/result/출력/최종학습데이터_서울.csv', thousands=',', index_col = 0)
raw

In [ ]:
raw['날짜'] = pd.to_datetime(raw["날짜"])
raw.info()

In [ ]:
raw.set_index("날짜", inplace=True)

In [ ]:
X = raw[['면적', '생산량', 'SPI1', 'EDI','면세유가', '수입중량(30일전)']]
Y = raw["도매가격"]

In [ ]:
Y = pd.DataFrame(Y)

In [ ]:
X_train = X.loc[:'2019-12-31']
X_test = X.loc['2020-01-01':]
y_train = Y.loc[:'2019-12-31']
y_test = Y.loc['2020-01-01':]

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = pd.DataFrame(X_train_scaled, columns=['면적', '생산량', 'SPI1', 'EDI','면세유가', '수입중량(30일전)'])
X_scaled

In [ ]:
scaler = MinMaxScaler()
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

In [ ]:
temp_X = dp(X_train_scaled)
temp_y = dp(y_train_scaled)

In [ ]:
step = 30
X_train_t = []
for i in range(0,len(temp_X)-step):
    X_train_t.append(temp_X[i:i+step])

In [ ]:
res = np.array(X_train_t)

In [ ]:
res_temp = []
for i in range(step,len(temp_y)):
    res_temp.append(temp_y[i])

res_y = np.array(res_temp)

In [ ]:
res_y = res_y.reshape(len(temp_y)-step)

In [ ]:
temp_test_x = dp(X_test_scaled)
temp_test_y = dp(y_test_scaled)
step = 30
X_test_t = []
for i in range(0,len(temp_test_x)-step):
    X_test_t.append(temp_test_x[i:i+step])

y_test_t = []
for j in range(step,len(temp_test_y)):
    y_test_t.append(temp_test_y[j])
    

X_test_res = np.array(X_test_t)
Y_test_res = np.array(y_test_t)
Y_test_res = Y_test_res.reshape(len(temp_test_y)-step)

In [ ]:
model = Sequential()
model.add(GRU(200, input_shape = (30, 6),recurrent_activation = 'tanh',return_sequences= True))
model.add(Dense(1))
model.add(GRU(100, input_shape = (30, 6),activation = 'tanh', return_sequences= True))
model.add(GRU(50, input_shape = (30, 6),activation = 'tanh', dropout=0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError("RMSE")])
model.summary()

In [ ]:
model.fit(res, res_y, epochs=50, batch_size = 4, verbose=1)

In [ ]:
y_pred = model.predict(X_test_res)

In [ ]:
def scoring(y_true, y_pred) :
    r2 = round(metrics.r2_score(y_true, y_pred),3)
    corr = round(np.corrcoef(y_true, y_pred)[0,1], 3)
    mape = round(metrics.mean_absolute_percentage_error(y_true, y_pred) * 100, 3)
    rmse = round(metrics.mean_squared_error(y_true, y_pred, squared = False), 3)
    
    df = pd.DataFrame({
        'R2' : r2,
        "Corr" : corr,
        "RMSE" : rmse,
        "MAPE" : mape
    }, index=[0])
    return df

In [ ]:
Y_test_res = Y_test_res.reshape(-1,1)
y_pred = y_pred.reshape(-1,1)
y_pred_real = scaler.inverse_transform(y_pred)
y_test_real = scaler.inverse_transform(Y_test_res)
y_pred_real = y_pred_real.reshape(336)
y_test_real = y_test_real.reshape(336)
scoring(y_test_real,y_pred_real)

## 시각화

In [ ]:
X_test_edit = X_test.reset_index()
X_test_edit = X_test_edit[:336]
X_test_edit['정답'] = y_test_real
X_test_edit['예측'] = y_pred_real
X_test_graph = X_test_edit.drop(['면적','SPI1', 'EDI', '면세유가', '수입중량(30일전)', '생산량'], axis = 1)

In [ ]:
X_test_edit['정답'] = y_test_real

In [ ]:
X_test_edit['예측'] = y_pred_real

In [ ]:
X_test_graph = X_test_edit.drop(['면적','SPI1', 'EDI', '면세유가', '수입중량(30일전)', '생산량'], axis = 1)

In [ ]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test_graph['날짜'], y=X_test_graph['정답'], name='Answer', mode='lines',
                         line={'width':6,
                              'color':'#11653F'}))
fig.add_trace(go.Scatter(x=X_test_graph['날짜'], y=X_test_graph['예측'], name='Prediction', mode='lines',
                         line={'width':6,
                               'color':'#E97D42'}))
fig.update_layout(template = 'seaborn',
                  paper_bgcolor='#F2EFE6',
                  plot_bgcolor='#F2EFE6',
                  xaxis={'title': {'text': '연도'}, # x축 라벨 추가, 그리드 숨김
                                   'showgrid':False},
                yaxis={'title': {'text': '서울_도매가격'}, # y축 라벨 추가
                                   'showgrid':False}
                 )
fig.show()